In [1]:
import numpy as np
import os
import tensorflow as tf

from datasets import imagenet
from tensorflow.contrib.slim.nets import inception
import inception_preprocessing
from matplotlib import pyplot as plt

slim = tf.contrib.slim
#batch_size = 3
image_size = inception.inception_v3.default_image_size
names = imagenet.create_readable_names_for_imagenet_labels()

img_list = ['/home/tiago/Desktop/img.jpeg', '/home/tiago/Desktop/img2.jpeg', '/home/tiago/Desktop/img3.jpeg']

In [2]:
with tf.Graph().as_default():
    filename_queue = tf.train.string_input_producer(img_list, shuffle=False)

    reader = tf.WholeFileReader()
    key, value = reader.read(filename_queue)
    image = tf.image.decode_jpeg(value, channels=3)
    processed_image = inception_preprocessing.preprocess_image(image, image_size, image_size, is_training=False)
    processed_images = tf.expand_dims(processed_image, 0)

    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v3_arg_scope()):
        logits, _ = inception.inception_v3(processed_images, num_classes=1001, is_training=False)
    probabilities = tf.nn.softmax(logits)

    init_fn = slim.assign_from_checkpoint_fn('/home/tiago/PycharmProjects/DeepTrack/inception_v3.ckpt', slim.get_model_variables('InceptionV3'))

    with tf.Session() as sess:
        init_fn(sess)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        for i in range(3):
            prb = probabilities.eval()

            argmax = np.argmax(prb)
            print('Probability %0.2f%% => [%s]' % (prb[argmax], names[argmax]))
            #plt.imshow(img)
            #plt.show()
            #np_image, probabilities = sess.run([image, probabilities])
            #probabilities = probabilities[0, 0:]
            #classification = np.argmax(probabilities)
            #print('Probability %0.2f%% => [%s]' % (probabilities[classification], names[classification]))
            #print(np_image.shape)
            #plt.imshow(np_image)
            #plt.show()

        coord.request_stop()
        coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueMany[Tcomponents=[DT_STRING], _class=["loc:@input_producer"], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/Identity)]]

Caused by op 'input_producer/input_producer_EnqueueMany', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.4/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance

IndexError: index 147 is out of bounds for axis 0 with size 1

In [3]:
prb

array([[ 0.00020923,  0.00017578,  0.00029098, ...,  0.00015783,
         0.00022501,  0.00025338]], dtype=float32)

In [4]:
prb[0]

array([ 0.00020923,  0.00017578,  0.00029098, ...,  0.00015783,
        0.00022501,  0.00025338], dtype=float32)

In [5]:
prb[0][0]

0.00020922985